# Fine Tuning Inception

In [1]:
import tensorflow as tf

import dataset_big as dataset
from inception_v4 import inception_v4, inception_v4_arg_scope
import tfutil as tfu

Inception expects 299x299 pixel images as input:

In [2]:
dataset.image_dim()

(299, 299)

In [3]:
def inception_inference_op(images, reg_terms=None, train=True, share=False):
    with tf.contrib.slim.arg_scope(inception_v4_arg_scope()):
        _, end_points = inception_v4(
            inputs=images,
            num_classes=1001,
            create_aux_logits=False,
            reuse=share,
            scope=tf.get_variable_scope(),
            is_training=train,
            dropout_keep_prob=1.0,
        )
    x = end_points['PreLogitsFlatten']
    # now create inference on bottlenecks
    with tf.variable_scope('inception_lm', reuse=share):
        keep_prob = 0.5 if train else 1.0
        h = tf.nn.dropout(x, keep_prob=keep_prob)
        h = tfu.fc_op(h, channels_in=1536, channels_out=1, name='out', relu=False)
        
    return h

In [4]:
NAME = 'inception'

args = {
    'name': NAME,
    'inference_op': inception_inference_op,
    'reg_terms': None,
    'inputs': dataset.inputs
}

training_args = {
    **args,
    'optimizer': tf.train.AdamOptimizer,
}

In [5]:
tfu.run_cleanup(name=NAME)
tfu.run_setup(name=NAME)

In [6]:
tfu.run_training(
    learning_rate=1e-4,
    num_epochs=25,
    **training_args
);

KeyboardInterrupt: 

In [ ]:
tf.get_v

In [7]:
tfu.run_eval(**args)

Train Accuracy: 99.5%
Validation Accuracy: 99.3%
Test Accuracy: 99.7%
Train Loss: 0.037
Validation Loss: 0.037
Test Loss: 0.033


In [8]:
tfu.run_prediction(**args, clip=True)
tfu.run_prediction(**args, clip=False)

Wrote predictions to ./data/inception_lm_clipped.csv
Wrote predictions to ./data/inception_lm.csv
